### below load dependency pkgs. and then create manually a new directory under experiments like base_model having params.json. this json file contains hyperparameter related to model, user can tweak these parameters for finding best model. and based on new name , change below model_dir path.

In [1]:
import argparse
import logging
import os

import numpy as np
import torch
from torch.autograd import Variable
import torch.optim as optim
from tqdm import trange
import random

# dependancy packages.
import utils
import model.net_w2v as net
from model.data_loader import DataLoader


data_dir='data/small'
model_dir='experiments/base_model'
word2vec_dir='experiments/word2vec' # keep GoogleNews-vectors-negative300.bin inside this directory.

# ## Let's use CoNLL 2002 data to build a NER system
# 
# CoNLL2002 corpus is available in NLTK. We use Spanish data.

In [2]:
import nltk
from sklearn.model_selection import cross_val_predict
nltk.corpus.conll2002.fileids()

train_sents = list(nltk.corpus.conll2002.iob_sents('esp.train'))
train_sents = [val for val in train_sents if len(val)!=0]
train_sents = train_sents[:5000]
test_sents = list(nltk.corpus.conll2002.iob_sents('esp.testb'))
test_sents = [val for val in test_sents if len(val)!=0]
test_sents = test_sents[:1000]

print(len(train_sents))
print(len(test_sents))


5000
1000


In [3]:
tokens_word2vec_list = []
tr_tokens_list = []
tr_pos_list = []
tr_labels_list = []
ts_tokens_list = []
ts_pos_list = []
ts_labels_list = []

for sents in train_sents:
    tokens = tuple(i[0].lower() for i in sents)
    pos_tags = [i[1] for i in sents]
    labels = tuple(i[2] for i in sents)
    tokens_word2vec_list.append(tokens)
    tr_tokens_list.append(tokens)
    tr_pos_list.append(pos_tags)
    tr_labels_list.append(labels)

for sents in test_sents:
    tokens = tuple(i[0].lower() for i in sents)
    pos_tags = [i[1] for i in sents]
    labels = tuple(i[2] for i in sents)
    tokens_word2vec_list.append(tokens)
    ts_tokens_list.append(tokens)
    ts_pos_list.append(pos_tags)
    ts_labels_list.append(labels)

print(tr_tokens_list[0])
print(tr_pos_list[:4])
print(tr_labels_list[0])

('melbourne', '(', 'australia', ')', ',', '25', 'may', '(', 'efe', ')', '.')
[['NP', 'Fpa', 'NP', 'Fpt', 'Fc', 'Z', 'NC', 'Fpa', 'NC', 'Fpt', 'Fp'], ['Fg'], ['DA', 'NC', 'AQ', 'SP', 'NC', 'Fc', 'VMI', 'NC', 'Fc', 'VMI', 'RG', 'DA', 'NC', 'SP', 'VMN', 'NC', 'SP', 'VMN', 'SP', 'NC', 'AQ', 'AQ', 'RG', 'SP', 'DI', 'NC', 'SP', 'NC', 'PR', 'VMI', 'DA', 'NC', 'SP', 'DA', 'NC', 'AQ', 'SP', 'DA', 'NC', 'Fp'], ['DA', 'NC', 'SP', 'NC', 'AQ', 'VMI', 'NC', 'RG', 'SP', 'CS', 'DI', 'NC', 'SP', 'NC', 'AQ', 'SP', 'NC', 'SP', 'NC', 'Fpa', 'NP', 'Fpt', 'P0', 'VMS', 'AQ', 'SP', 'VMN', 'DI', 'NC', 'AQ', 'CC', 'VMN', 'DA', 'NC', 'SP', 'DA', 'NC', 'SP', 'DA', 'NC', 'SP', 'CS', 'DA', 'NC', 'PR', 'PP', 'VMI', 'VMI', 'VAN', 'VMP', 'NC', 'SP', 'DA', 'VMP', 'SP', 'NC', 'SP', 'DA', 'NC', 'AQ', 'Fp']]
('B-LOC', 'O', 'B-LOC', 'O', 'O', 'O', 'O', 'O', 'B-ORG', 'O', 'O')


In [4]:
from gensim.models import Word2Vec
sentences = tokens_word2vec_list#[["cat", "say", "meow"], ["dog", "say", "woof"]]

emb_model = Word2Vec(min_count=1)
emb_model.build_vocab(sentences)  # prepare the model vocabulary
emb_model.train(sentences, total_examples=emb_model.corpus_count, epochs=emb_model.iter)  # train word vectors

/home/adzuser/user_achyuta/venv3.6/lib/python3.6/site-packages/ipykernel_launcher.py:6: DeprecationWarning: Call to deprecated `iter` (Attribute will be removed in 4.0.0, use self.epochs instead).
  


(637027, 960020)

In [5]:
emb_model['australia'].astype(np.float64)

/home/adzuser/user_achyuta/venv3.6/lib/python3.6/site-packages/ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  """Entry point for launching an IPython kernel.


array([ 0.02549249, -0.06803773,  0.0823159 ,  0.06937288,  0.06129755,
       -0.04450702, -0.00695999, -0.04581102, -0.04970881,  0.02325225,
        0.04840021,  0.0205205 ,  0.05564537,  0.01700225,  0.04521008,
        0.01107221,  0.08158567, -0.01456591, -0.06555842,  0.15479001,
       -0.04996794, -0.00248865,  0.00048141, -0.02581766, -0.14801976,
        0.12446092, -0.07381675,  0.00890755, -0.05540852, -0.15583819,
       -0.05111098, -0.02409653, -0.04719809,  0.01424056, -0.00910902,
       -0.13002925,  0.06916769,  0.10206576, -0.04936216,  0.0386195 ,
        0.01611323, -0.00957582,  0.02712297,  0.02905316,  0.06691138,
        0.03186531,  0.04966923, -0.08750893, -0.05427027,  0.01550188,
        0.04272985,  0.07804484, -0.06363418,  0.01143038,  0.03376096,
       -0.02138905, -0.01400748, -0.09824818, -0.05316613, -0.06413367,
        0.11202486, -0.00245094,  0.02940268, -0.03232801, -0.05148359,
       -0.04678078, -0.01598695,  0.03286493,  0.03945813,  0.07

In [6]:
"""Build vocabularies of words and tags from datasets"""

import argparse
from collections import Counter
import json
import os


parser = {'min_count_word':1,'min_count_tag':1,'data_dir':'data/small'}#argparse.ArgumentParser()
#parser.add_argument('--min_count_word', default=1, help="Minimum count for words in the dataset", type=int)
#parser.add_argument('--min_count_tag', default=1, help="Minimum count for tags in the dataset", type=int)
#parser.add_argument('--data_dir', default='data/small', help="Directory containing the dataset")

# Hyper parameters for the vocab
PAD_WORD = '<pad>'
PAD_TAG = 'O'
UNK_WORD = 'UNK'


def save_vocab_to_txt_file(vocab, txt_path):
    """Writes one token per line, 0-based line id corresponds to the id of the token.

    Args:
        vocab: (iterable object) yields token
        txt_path: (stirng) path to vocab file
    """
    with open(txt_path, "w") as f:
        for token in vocab:
            f.write(token + '\n')
            

def save_dict_to_json(d, json_path):
    """Saves dict to json file

    Args:
        d: (dict)
        json_path: (string) path to json file
    """
    with open(json_path, 'w') as f:
        d = {k: v for k, v in d.items()}
        json.dump(d, f, indent=4)


def update_vocab(tokens_list, vocab):
    """Update word and tag vocabulary from dataset

    Args:
        txt_path: (string) path to file, one sentence per line
        vocab: (dict or Counter) with update method

    Returns:
        dataset_size: (int) number of elements in the dataset
    """
    for i, tokens in enumerate(tokens_list):
        vocab.update(tokens)

    return i + 1


if __name__ == '__main__':
    #args = parser.parse_args()

    # Build word vocab with train and test datasets
    print("Building word vocabulary...")
    words = Counter()
    size_train_sentences = update_vocab(tr_tokens_list, words)
    size_dev_sentences = update_vocab(ts_tokens_list, words)
    size_test_sentences = update_vocab(ts_tokens_list, words)
    print("- done.")

    # Build tag vocab with train and test datasets
    print("Building tag vocabulary...")
    tags = Counter()
    size_train_tags = update_vocab(tr_labels_list, tags)
    size_dev_tags = update_vocab(ts_labels_list, tags)
    size_test_tags = update_vocab(ts_labels_list, tags)
    print("- done.")

    # Assert same number of examples in datasets
    assert size_train_sentences == size_train_tags
    assert size_dev_sentences == size_dev_tags
    assert size_test_sentences == size_test_tags

    # Only keep most frequent tokens
    words = [tok for tok, count in words.items() if count >= parser['min_count_word']]
    tags = [tok for tok, count in tags.items() if count >= parser['min_count_tag']]

    # Add pad tokens
    if PAD_WORD not in words: words.append(PAD_WORD)
    if PAD_TAG not in tags: tags.append(PAD_TAG)
    
    # add word for unknown words 
    words.append(UNK_WORD)

    # Save vocabularies to file
    print("Saving vocabularies to file...")
    save_vocab_to_txt_file(words, os.path.join(parser['data_dir'], 'words.txt'))
    save_vocab_to_txt_file(tags, os.path.join(parser['data_dir'], 'tags.txt'))
    print("- done.")

    # Save datasets properties in json file
    sizes = {
        'train_size': size_train_sentences,
        'dev_size': size_dev_sentences,
        'test_size': size_test_sentences,
        'vocab_size': len(words),
        'number_of_tags': len(tags),
        'pad_word': PAD_WORD,
        'pad_tag': PAD_TAG,
        'unk_word': UNK_WORD
    }
    
    save_dict_to_json(sizes, os.path.join(parser['data_dir'], 'dataset_params.json'))

    # Logging sizes
    to_print = "\n".join("- {}: {}".format(k, v) for k, v in sizes.items())
    print("Characteristics of the dataset:\n{}".format(to_print))

Building word vocabulary...
- done.
Building tag vocabulary...
- done.
Saving vocabularies to file...
- done.
Characteristics of the dataset:
- train_size: 5000
- dev_size: 1000
- test_size: 1000
- vocab_size: 19952
- number_of_tags: 9
- pad_word: <pad>
- pad_tag: O
- unk_word: UNK


In [7]:
'''
from sklearn.preprocessing import OneHotEncoder
enc = OneHotEncoder()
enc.fit(pos_list)#([[0, 0, 3], [1, 1, 0], [0, 2, 1], [1, 0, 2]])   
print(enc.n_values_)
print(enc.feature_indices_)
print(enc.transform([('NP', 'Fpa', 'NP', 'Fpt', 'Fc', 'Z', 'NC', 'Fpa', 'NC', 'Fpt', 'Fp')]).toarray())
'''
from numpy import argmax
import itertools

# define input string
data = ['NP', 'Fpa', 'NP', 'Fpt', 'Fc', 'Z', 'NC', 'Fpa', 'NC', 'Fpt', 'Fp']

# define universe of possible input values
unq_pos_list = list(itertools.chain.from_iterable(tr_pos_list+ts_pos_list))
unq_pos_list = list(set(unq_pos_list))
print(unq_pos_list)

# define a mapping of chars to integers
char_to_int = dict((c, i) for i, c in enumerate(unq_pos_list))
int_to_char = dict((i, c) for i, c in enumerate(unq_pos_list))

def onehot_encode(data,unq_pos_list,char_to_int):
    # integer encode input data
    integer_encoded = [char_to_int[pos] for pos in data]
    print(integer_encoded)

    # one hot encode
    onehot_encoded = list()
    for value in integer_encoded:
	    letter = [0 for _ in range(len(unq_pos_list))]
	    letter[value] = 1
	    onehot_encoded.append(letter)
    #print(onehot_encoded)
    return onehot_encoded

onehot_encoded = onehot_encode(data,unq_pos_list,char_to_int)
print(onehot_encoded)

# invert encoding
inverted = int_to_char[argmax(onehot_encoded[0])]
print(inverted)

['PD', 'RG', 'VMS', 'SP', 'VMG', 'DN', 'Fpt', 'Fp', 'Fc', 'RN', 'VSP', 'VAN', 'PR', 'VSM', 'Fz', 'Fit', 'NC', 'P0', 'PP', 'Fia', 'DA', 'AQ', 'Y', 'Fx', 'DD', 'Ft', 'Fs', 'DI', 'VAP', 'VSG', 'VSN', 'AO', 'Fat', 'PX', 'VMM', 'DP', 'VMI', 'VMN', 'I', 'Fpa', 'Fe', 'VSI', 'PT', 'CC', 'VAS', 'Faa', 'VAI', 'Fg', 'VSS', 'Fh', 'PN', 'Z', 'PI', 'DT', 'CS', 'NP', 'VMP', 'Fd']
[55, 39, 55, 6, 8, 51, 16, 39, 16, 6, 7]
[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0], [0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0

# train & evaluation wrappers.

In [8]:
class data_loader(object):
    def data_iterator(self, data_dir, data, params, shuffle=False):
        """
        Returns a generator that yields batches data with labels. Batch size is params.batch_size. Expires after one
        pass over the data.

        Args:
            data: (dict) contains data which has keys 'data', 'labels' and 'size'
            params: (Params) hyperparameters of the training process.
            shuffle: (bool) whether the data should be shuffled

        Yields:
            batch_data: (Variable) dimension batch_size x seq_len with the sentence data
            batch_labels: (Variable) dimension batch_size x seq_len with the corresponding labels

        """

        # make a list that decides the order in which we go over the data- this avoids explicit shuffling of data
        order = list(range(data['size']))
        if shuffle:
            random.seed(230)
            random.shuffle(order)

        # loading dataset_params
        json_path = os.path.join(data_dir, 'dataset_params.json')
        assert os.path.isfile(json_path), "No json file found at {}, run build_vocab.py".format(json_path)
        dataset_params = utils.Params(json_path)        

        # loading vocab (we require this to map words to their indices)
        vocab_path = os.path.join(data_dir, 'words.txt')
        vocab = {}
        with open(vocab_path) as f:
            for i, l in enumerate(f.read().splitlines()):
                vocab[l] = i

        # setting the indices for UNKnown words and PADding symbols
        self.unk_ind = vocab[dataset_params.unk_word]
        self.pad_ind = vocab[dataset_params.pad_word]
        
        tr_tokens_list_ind = []
        for sent_token_list in data['data']:
            s = [vocab[token] if token in vocab else self.unk_ind for token in sent_token_list]
            tr_tokens_list_ind.append(s)
        data['data'] = tr_tokens_list_ind
        

        # one pass over data
        for i in range((data['size']+1)//params.batch_size):
            # fetch sentences and tags
            batch_sentences = [data['data'][idx] for idx in order[i*params.batch_size:(i+1)*params.batch_size]]
            batch_tags = [data['labels'][idx] for idx in order[i*params.batch_size:(i+1)*params.batch_size]]

            # compute length of longest sentence in batch
            batch_max_len = max([len(s) for s in batch_sentences])

            # prepare a numpy array with the data, initialising the data with pad_ind and all labels with -1
            # initialising labels to -1 differentiates tokens with tags from PADding tokens
            batch_data = self.pad_ind*np.ones((len(batch_sentences), batch_max_len))
            batch_labels = -1*np.ones((len(batch_sentences), batch_max_len))
            
            #print("------------------------------")
            #print(np.array(batch_sentences).shape)
            #print(batch_data.shape)
            #print(batch_labels.shape)
            #print(np.array(batch_tags).shape)
            #print(batch_sentences)
            #print("===================")
            #print(batch_tags)

            # copy the data to the numpy array
            for j in range(len(batch_sentences)):
                cur_len = len(batch_sentences[j])
                #print(cur_len)
                #print(batch_data[j][:cur_len].shape)
                batch_data[j][:cur_len] = batch_sentences[j]
                batch_labels[j][:cur_len] = batch_tags[j]

            # since all data are indices, we convert them to torch LongTensors
            batch_data, batch_labels = torch.LongTensor(batch_data), torch.LongTensor(batch_labels)
            #print(type(batch_data))
            #print(type(batch_labels))
            #batch_data = batch_data.to(dtype=torch.long)
            #batch_labels = batch_labels.to(dtype=torch.long)

            # shift tensors to GPU if available
            if params.cuda:
                batch_data, batch_labels = batch_data.cuda(), batch_labels.cuda()

            # convert them to Variables to record operations in the computational graph
            batch_data, batch_labels = Variable(batch_data), Variable(batch_labels)
            #print(type(batch_data))
            #print(type(batch_labels))
            #print(batch_data.dtype)
            #print(batch_data[0].dtype)
            #print(batch_data[0])

            yield batch_data, batch_labels


def train(model, optimizer, loss_fn, data_iterator, metrics, params, num_steps):
    """Train the model on `num_steps` batches

    Args:
        model: (torch.nn.Module) the neural network
        optimizer: (torch.optim) optimizer for parameters of model
        loss_fn: a function that takes batch_output and batch_labels and computes the loss for the batch
        data_iterator: (generator) a generator that generates batches of data and labels
        metrics: (dict) a dictionary of functions that compute a metric using the output and labels of each batch
        params: (Params) hyperparameters
        num_steps: (int) number of batches to train on, each of size params.batch_size
    """

    # set model to training mode
    model.train()

    # summary for current training loop and a running average object for loss
    summ = []
    loss_avg = utils.RunningAverage()
    
    # Use tqdm for progress bar
    t = trange(num_steps) 
    for i in t:
        # fetch the next training batch
        train_batch, labels_batch = next(data_iterator)
        #print("-------------------------")
        #print(train_batch.shape)
        #print(labels_batch.shape)

        # compute model output and loss
        output_batch = model(train_batch)
        loss = loss_fn(output_batch, labels_batch)

        # clear previous gradients, compute gradients of all variables wrt loss
        optimizer.zero_grad()
        loss.backward()

        # performs updates using calculated gradients
        optimizer.step()

        # Evaluate summaries only once in a while
        if i % params.save_summary_steps == 0:
            # extract data from torch Variable, move to cpu, convert to numpy arrays
            output_batch = output_batch.data.cpu().numpy()
            labels_batch = labels_batch.data.cpu().numpy()

            # compute all metrics on this batch
            summary_batch = {metric:metrics[metric](output_batch, labels_batch)
                             for metric in metrics}
            summary_batch['loss'] = loss.data.item()
            summ.append(summary_batch)

        # update the average loss
        loss_avg.update(loss.data.item())
        t.set_postfix(loss='{:05.3f}'.format(loss_avg()))

    # compute mean of all metrics in summary
    metrics_mean = {metric:np.mean([x[metric] for x in summ]) for metric in summ[0]} 
    metrics_string = " ; ".join("{}: {:05.3f}".format(k, v) for k, v in metrics_mean.items())
    

def evaluate(model, loss_fn, data_iterator, metrics, params, num_steps):
    """Evaluate the model on `num_steps` batches.

    Args:
        model: (torch.nn.Module) the neural network
        loss_fn: a function that takes batch_output and batch_labels and computes the loss for the batch
        data_iterator: (generator) a generator that generates batches of data and labels
        metrics: (dict) a dictionary of functions that compute a metric using the output and labels of each batch
        params: (Params) hyperparameters
        num_steps: (int) number of batches to train on, each of size params.batch_size
    """

    # set model to evaluation mode
    model.eval()

    # summary for current eval loop
    summ = []

    # compute metrics over the dataset
    for _ in range(num_steps):
        # fetch the next evaluation batch
        data_batch, labels_batch = next(data_iterator)
        
        # compute model output
        output_batch = model(data_batch)
        loss = loss_fn(output_batch, labels_batch)

        # extract data from torch Variable, move to cpu, convert to numpy arrays
        output_batch = output_batch.data.cpu().numpy()
        labels_batch = labels_batch.data.cpu().numpy()

        # compute all metrics on this batch
        summary_batch = {metric: metrics[metric](output_batch, labels_batch)
                         for metric in metrics}
        summary_batch['loss'] = loss.data.item()
        summ.append(summary_batch)

    # compute mean of all metrics in summary
    metrics_mean = {metric:np.mean([x[metric] for x in summ]) for metric in summ[0]} 
    metrics_string = " ; ".join("{}: {:05.3f}".format(k, v) for k, v in metrics_mean.items())
    return metrics_mean

def train_and_evaluate(model, train_data, val_data, optimizer, loss_fn, metrics, params, model_dir, data_dir, restore_file=None):
    """Train the model and evaluate every epoch.

    Args:
        model: (torch.nn.Module) the neural network
        train_data: (dict) training data with keys 'data' and 'labels'
        val_data: (dict) validaion data with keys 'data' and 'labels'
        optimizer: (torch.optim) optimizer for parameters of model
        loss_fn: a function that takes batch_output and batch_labels and computes the loss for the batch
        metrics: (dict) a dictionary of functions that compute a metric using the output and labels of each batch
        params: (Params) hyperparameters
        model_dir: (string) directory containing config, weights and log
        restore_file: (string) optional- name of file to restore from (without its extension .pth.tar)
    """
    # reload weights from restore_file if specified
    if restore_file is not None:
        restore_path = os.path.join(args.model_dir, args.restore_file + '.pth.tar')
        utils.load_checkpoint(restore_path, model, optimizer)
        
    best_val_acc = 0.0

    for epoch in range(params.num_epochs):
        # Run one epoch
        # compute number of batches in one epoch (one full pass over the training set)
        num_steps = (params.train_size + 1) // params.batch_size
        dl = data_loader()
        train_data_iterator = dl.data_iterator(data_dir, train_data, params, shuffle=True)
        train(model, optimizer, loss_fn, train_data_iterator, metrics, params, num_steps)
            
        # Evaluate for one epoch on validation set
        num_steps = (params.val_size + 1) // params.batch_size
        val_data_iterator = dl.data_iterator(data_dir, val_data, params, shuffle=False)
        val_metrics = evaluate(model, loss_fn, val_data_iterator, metrics, params, num_steps)
        
        val_acc = val_metrics['accuracy']
        is_best = val_acc >= best_val_acc

        # Save weights
        utils.save_checkpoint({'epoch': epoch + 1,
                               'state_dict': model.state_dict(),
                               'optim_dict' : optimizer.state_dict()}, 
                               is_best=is_best,
                               checkpoint=model_dir)
            
        # If best_eval, best_save_path        
        if is_best:
            best_val_acc = val_acc
            
            # Save best val metrics in a json file in the model directory
            best_json_path = os.path.join(model_dir, "metrics_val_best_weights.json")
            utils.save_dict_to_json(val_metrics, best_json_path)

        # Save latest val metrics in a json file in the model directory
        last_json_path = os.path.join(model_dir, "metrics_val_last_weights.json")
        utils.save_dict_to_json(val_metrics, last_json_path)

# Train

In [9]:
restore_file=None

# Load the parameters from json file
json_path = os.path.join(model_dir, 'params.json')
assert os.path.isfile(json_path), "No json configuration file found at {}".format(json_path)
params = utils.Params(json_path)

# use GPU if available
params.cuda = torch.cuda.is_available()
    
# Set the random seed for reproducible experiments
torch.manual_seed(230)
if params.cuda: torch.cuda.manual_seed(230)
        
# Set the logger
utils.set_logger(os.path.join(model_dir, 'train.log'))

# Create the input data pipeline
logging.info("Loading the datasets...")
    
# load data
#data_loader = DataLoader(data_dir, params)
#data = data_loader.load_data(['train', 'val'], data_dir)
#train_data = data['train']
#val_data = data['val']
params.vocab_size=sizes['vocab_size']
params.number_of_tags = sizes['number_of_tags']

# add word2vec
'''
train_data = {'data':[[],[]],'labels':[[]],'size':10}
'''
tr_data = None
tr_pos = None
ts_data = None
ts_pos = None
def preprocess_data(tokens_list,pos_list):
    tr_data = []
    for v in tokens_list:
        tmp_tr_data = []
        for v_in in v:
            tmp_tr_data.append(emb_model[v_in.lower()].tolist()) #.astype(np.float64)
        tr_data.append(tmp_tr_data)
    
    tr_pos = [onehot_encode(v,unq_pos_list,char_to_int) for v in pos_list]
    return tr_data,tr_pos

tr_data,tr_pos = preprocess_data(tr_tokens_list,tr_pos_list)
ts_data,ts_pos = preprocess_data(ts_tokens_list,ts_pos_list)


# loading tags (we require this to map tags to their indices)
tags_path = os.path.join(data_dir, 'tags.txt')
tag_map = {}
with open(tags_path) as f:
    for i, t in enumerate(f.read().splitlines()):
        tag_map[t] = i

def preprocess_label(tag_map, labels_list):
    fnl_labels = []
    for label_sent in labels_list:
        tmp_label = []
        for label in label_sent:
            tmp_label.append(tag_map[label])
        fnl_labels.append(tmp_label)
    return fnl_labels

tr_label_list = preprocess_label(tag_map, tr_labels_list)
ts_label_list = preprocess_label(tag_map, ts_labels_list)

tr_tokens_list_1 = []
tr_label_list_1 = []
for i,j in zip(tr_tokens_list,tr_label_list):
    if len(i) != len(j):
        print("not equal length : ")
        continue
    tr_tokens_list_1.append(i)
    tr_label_list_1.append(j)

ts_tokens_list_1 = []
ts_label_list_1 = []
for i,j in zip(ts_tokens_list,ts_label_list):
    if len(i) != len(j):
        print("not equal length : ")
        continue
    ts_tokens_list_1.append(i)
    ts_label_list_1.append(j)

train_data = {'data':tr_tokens_list_1,'labels':tr_label_list_1,'size':len(tr_tokens_list_1)}
val_data = {'data':ts_tokens_list_1,'labels':ts_label_list_1,'size':len(ts_tokens_list_1)}

# specify the train and val dataset sizes
params.train_size = train_data['size']
params.val_size = val_data['size']

logging.info("- done.")

# loading vocab (we require this to map words to their indices)
vocab_path = os.path.join(data_dir, 'words.txt')
vocab = {}
with open(vocab_path) as f:
    for i, l in enumerate(f.read().splitlines()):
        vocab[l] = i

weight = np.zeros((len(vocab),params.embedding_dim))
vocab_inv = {y:x for x,y in vocab.items()}
for ind,val in vocab_inv.items():
    try:
        weight[ind][:]= emb_model[val]
    except:
        print("exception word not have embeddings : ",val)

# Define the model and optimizer
model = net.Net(params).cuda() if params.cuda else net.Net(params)
#weight = torch.FloatTensor([[1, 2.3, 3], [4, 5.1, 6.3]])
model.init_word_embeddings(torch.FloatTensor(weight).cuda())
optimizer = optim.Adam(model.parameters(), lr=params.learning_rate)
    
# fetch loss function and metrics
loss_fn = net.loss_fn
metrics = net.metrics

# Train the model
logging.info("Starting training for {} epoch(s)".format(params.num_epochs))
train_and_evaluate(model, train_data, val_data, optimizer, loss_fn, metrics, params, model_dir, data_dir,
                    restore_file)

Loading the datasets...
/home/adzuser/user_achyuta/venv3.6/lib/python3.6/site-packages/ipykernel_launcher.py:42: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).


[55, 39, 55, 6, 8, 51, 16, 39, 16, 6, 7]
[47]
[20, 16, 21, 3, 16, 8, 36, 16, 8, 36, 1, 20, 16, 3, 37, 16, 3, 37, 3, 16, 21, 21, 1, 3, 27, 16, 3, 16, 12, 36, 20, 16, 3, 20, 16, 21, 3, 20, 16, 7]
[20, 16, 3, 16, 21, 36, 16, 1, 3, 54, 27, 16, 3, 16, 21, 3, 16, 3, 16, 39, 55, 6, 17, 2, 21, 3, 37, 27, 16, 21, 43, 37, 20, 16, 3, 20, 16, 3, 20, 16, 3, 54, 20, 16, 12, 18, 36, 36, 11, 56, 16, 3, 20, 56, 3, 16, 3, 20, 16, 21, 7]
[24, 16, 21, 36, 27, 16, 3, 16, 8, 16, 3, 20, 12, 46, 10, 56, 3, 1, 3, 27, 16, 3, 16, 8, 43, 36, 16, 3, 50, 3, 16, 43, 34, 1, 56, 43, 1, 34, 8, 16, 12, 36, 3, 16, 3, 16, 43, 16, 21, 7]
[3, 35, 16, 8, 20, 16, 21, 3, 16, 8, 16, 21, 8, 36, 54, 9, 46, 52, 12, 2, 54, 20, 16, 21, 3, 16, 41, 21, 7]
[55, 36, 54, 3, 20, 16, 21, 3, 20, 16, 8, 3, 20, 12, 17, 36, 20, 16, 21, 8, 41, 21, 12, 20, 16, 48, 56, 1, 4, 3, 16, 20, 16, 21, 3, 20, 16, 7]
[20, 21, 16, 3, 16, 17, 36, 1, 20, 16, 3, 20, 16, 21, 3, 27, 16, 3, 16, 56, 1, 3, 20, 16, 3, 16, 8, 21, 3, 16, 3, 16, 7]
[16, 8, 51, 16, 39, 

[20, 16, 3, 20, 16, 21, 3, 37, 16, 8, 21, 16, 8, 36, 54, 40, 3, 16, 36, 37, 3, 27, 16, 21, 40, 8, 16, 3, 20, 16, 12, 17, 36, 8, 43, 36, 54, 40, 20, 16, 43, 21, 41, 37, 3, 16, 21, 40, 7]
[20, 21, 16, 40, 1, 36, 54, 17, 36, 16, 8, 27, 16, 1, 21, 8, 3, 16, 3, 16, 40, 36, 8, 43, 36, 54, 40, 20, 16, 41, 20, 16, 21, 3, 16, 3, 20, 16, 21, 8, 54, 16, 3, 18, 41, 21, 20, 16, 3, 16, 3, 16, 43, 20, 16, 21, 3, 16, 3, 20, 16, 40, 8, 3, 27, 27, 16, 7]
[36, 1, 36, 20, 21, 16, 3, 27, 16, 3, 16, 3, 20, 16, 21, 3, 16, 3, 16, 3, 37, 16, 21, 3, 20, 16, 3, 20, 51, 16, 3, 16, 56, 3, 27, 20, 16, 21, 7]
[16, 8, 51, 16, 39, 16, 6, 7]
[47]
[20, 16, 3, 16, 21, 36, 1, 27, 16, 21, 3, 16, 3, 37, 3, 20, 16, 3, 20, 16, 3, 16, 8, 54, 3, 35, 16, 36, 4, 16, 9, 17, 2, 20, 16, 3, 21, 3, 20, 16, 21, 7]
[3, 16, 3, 27, 16, 8, 16, 36, 54, 40, 20, 16, 3, 16, 3, 20, 16, 3, 16, 36, 3, 20, 16, 3, 27, 16, 3, 16, 8, 3, 20, 12, 2, 1, 20, 16, 54, 20, 16, 12, 2, 56, 20, 16, 21, 43, 3, 20, 16, 21, 40, 7]
[3, 20, 16, 21, 3, 16, 8, 16, 21

[3, 20, 16, 21, 3, 16, 3, 20, 16, 21, 16, 3, 16, 8, 20, 16, 16, 43, 20, 16, 21, 17, 36, 3, 20, 16, 21, 8, 36, 21, 8, 43, 36, 3, 20, 16, 3, 27, 16, 3, 5, 16, 3, 20, 16, 3, 5, 16, 7]
[3, 16, 3, 16, 8, 20, 16, 41, 21, 3, 20, 16, 3, 21, 8, 16, 21, 21, 8, 43, 3, 16, 3, 16, 18, 36, 20, 16, 3, 5, 16, 8, 21, 16, 7]
[1, 3, 16, 21, 8, 20, 16, 12, 36, 3, 20, 16, 21, 36, 21, 3, 27, 21, 16, 3, 16, 3, 16, 8, 12, 36, 16, 3, 27, 16, 21, 16, 3, 16, 7]
[20, 16, 36, 1, 27, 16, 3, 16, 3, 20, 16, 3, 16, 21, 43, 36, 20, 16, 3, 16, 3, 20, 16, 21, 7]
[3, 35, 16, 3, 16, 8, 16, 21, 17, 36, 3, 20, 16, 3, 16, 43, 3, 20, 16, 3, 16, 3, 16, 21, 8, 21, 21, 21, 43, 16, 36, 1, 8, 43, 36, 20, 16, 21, 3, 16, 43, 20, 16, 21, 16, 3, 20, 16, 3, 20, 16, 7]
[20, 16, 3, 16, 43, 35, 16, 36, 1, 5, 16, 21, 8, 12, 41, 16, 21, 8, 21, 3, 16, 3, 20, 16, 21, 3, 16, 7]
[16, 8, 51, 16, 39, 16, 6, 7]
[47]
[20, 16, 1, 43, 16, 3, 20, 16, 8, 21, 16, 8, 36, 1, 54, 20, 16, 36, 40, 27, 20, 16, 21, 40, 3, 37, 43, 37, 20, 16, 3, 20, 16, 21, 16, 

[16, 21, 3, 16, 21, 8, 4, 20, 16, 3, 20, 16, 7]
[36, 16, 7]
[47]
[21, 3, 20, 21, 8, 3, 16, 3, 27, 16, 21, 21, 7]
[3, 20, 16, 36, 1, 21, 8, 3, 27, 16, 21, 3, 20, 16, 7]
[16, 3, 27, 16, 3, 16, 3, 20, 21, 7]
[16, 3, 16, 43, 3, 21, 16, 7]
[16, 21, 3, 16, 21, 7]
[16, 21, 7]
[47]
[3, 20, 16, 21, 36, 1, 21, 3, 27, 16, 3, 20, 16, 3, 20, 16, 8, 54, 54, 3, 20, 16, 21, 46, 16, 21, 7]
[1, 36, 37, 27, 16, 1, 21, 3, 20, 16, 3, 52, 3, 20, 16, 7]
[16, 21, 7]
[16, 3, 16, 43, 3, 21, 16, 7]
[16, 21, 21, 3, 16, 7]
[36, 7]
[47]
[16, 21, 3, 16, 3, 16, 3, 20, 1, 3, 20, 16, 8, 3, 27, 16, 1, 21, 3, 20, 1, 7]
[16, 21, 7]
[16, 21, 3, 16, 43, 21, 3, 16, 43, 3, 21, 16, 7]
[16, 21, 21, 3, 16, 7]
[21, 7]
[47]
[21, 3, 16, 3, 16, 21, 12, 36, 3, 20, 16, 3, 1, 21, 7]
[16, 3, 16, 7]
[16, 3, 16, 24, 16, 3, 16, 3, 16, 3, 16, 7]
[16, 7]
[47]
[3, 20, 16, 21, 8, 36, 1, 21, 8, 3, 16, 21, 3, 16, 43, 16, 21, 21, 8, 54, 54, 3, 20, 16, 21, 3, 20, 16, 46, 16, 21, 3, 16, 21, 43, 21, 8, 12, 36, 30, 1, 21, 7]
[16, 3, 16, 43, 3, 21, 16

...

Checkpoint Directory exists! 


  0%|          | 0/1000 [00:00<?, ?it/s]

Checkpoint Directory exists! 


100%|██████████| 1000/1000 [00:10<00:00, 94.63it/s, loss=0.597]


Checkpoint Directory exists! 


In [12]:
#train_and_evaluate(model, train_data, val_data, optimizer, loss_fn, metrics, params, model_dir, data_dir,restore_file)

# Evaluation

In [10]:
"""
    Evaluate the model on the test set.
"""

restore_file='best'

# Load the parameters
json_path = os.path.join(model_dir, 'params.json')
assert os.path.isfile(json_path), "No json configuration file found at {}".format(json_path)
params = utils.Params(json_path)

# use GPU if available
params.cuda = torch.cuda.is_available()     # use GPU is available

# Set the random seed for reproducible experiments
torch.manual_seed(230)
if params.cuda: torch.cuda.manual_seed(230)

params.vocab_size=sizes['vocab_size']
params.number_of_tags = sizes['number_of_tags']

# load data
#data_loader = DataLoader(data_dir, params)
#data = data_loader.load_data(['test'], data_dir)
#test_data = data['test']

test_data = val_data

# specify the test set size
params.test_size = test_data['size']
#test_data_iterator = data_loader.data_iterator(test_data, params)
dll = data_loader()
test_data_iterator = dll.data_iterator(data_dir, test_data, params)

# Define the model
model = net.Net(params).cuda() if params.cuda else net.Net(params)
    
loss_fn = net.loss_fn
metrics = net.metrics

# Reload weights from the saved file
utils.load_checkpoint(os.path.join(model_dir, restore_file + '.pth.tar'), model)

# Evaluate
num_steps = (params.test_size + 1) // params.batch_size
test_metrics = evaluate(model, loss_fn, test_data_iterator, metrics, params, num_steps)
save_path = os.path.join(model_dir, "metrics_test_{}.json".format(restore_file))
utils.save_dict_to_json(test_metrics, save_path)
print(test_metrics)

{'accuracy': 0.8660236441172557, 'loss': 0.7808608418703079}
